In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\train.csv')
test=pd.read_csv('F:\\study\\ml\\DataSet\\House_Prices\\test.csv')

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
train['GarageType'].unique()

delete outlier data and drop id

In [ ]:
plt.scatter(train['GrLivArea'],train['SalePrice'])
#plt.show()

In [ ]:
train=train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [ ]:
plt.scatter(train['GrLivArea'],train['SalePrice'])
#plt.show()

In [ ]:
train.head()

In [ ]:
train_id=train['Id']
test_id=test['Id']

In [ ]:
train.drop('Id',axis=1,inplace=True)
test.drop('Id',axis=1,inplace=True)

- fit norm dist plot

In [ ]:
sns.distplot(train['SalePrice'],fit=norm)
#plt.show()

- - plot Q-Q 
- double check skew >0

In [ ]:
stats.probplot(train['SalePrice'],plot=plt)
#plt.show()

 - log transform target variable

In [ ]:
sns.distplot(np.log1p(train['SalePrice']),fit=norm)
#plt.show()

In [ ]:
sns.distplot(np.log(1/train['SalePrice']),fit=norm)
#plt.show()

In [ ]:
sns.distplot(np.sqrt(train['SalePrice']),fit=norm)
#plt.show()

In [ ]:
train['SalePrice']=np.log1p(train['SalePrice'])
(mu,sigma)=norm.fit(train['SalePrice'])
stats.probplot(train['SalePrice'],plot=plt)
#plt.show()

- features engineing

In [ ]:
ntrain=train.shape[0]
ntest=test.shape[0]
y_train=train.SalePrice.values
all_data=pd.concat((train,test)).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)
print('all data shape:',all_data.shape)

- Missing Data

In [ ]:
all_data_na=(all_data.isnull().sum()/len(all_data)) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data=pd.DataFrame({'Missing Ratio': all_data_na})
missing_data.head(20)

In [ ]:
plt.subplots(figsize=(12,10))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index,y=all_data_na)
plt.xlabel('Features',fontsize=15)
plt.ylabel('Precent of missing value',fontsize=15)
plt.title('Precent of miss data by feature',fontsize=15)
plt.show()

- Data Correlation

In [ ]:
corrmat=train.corr()
sns.heatmap(corrmat,vmax=0.9,square=True)

- imputing missing values

In [ ]:
all_data['LotFrontage'].head()

In [ ]:
all_data['Neighborhood'].head()

In [ ]:
for i in all_data.groupby('Neighborhood')['LotFrontage']:
    print(i)

In [ ]:
all_data['PoolQC']=all_data['PoolQC'].fillna('None')
all_data['MiscFeature']=all_data['MiscFeature'].fillna('None')
all_data['Alley']=all_data['Alley'].fillna('None')
all_data['Fence']=all_data['Fence'].fillna('None')
all_data['FireplaceQu']=all_data['FireplaceQu'].fillna('None')
all_data['LotFrontage']=all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))

In [ ]:
for col in ['GarageType','GarageFinish','GarageQual','GarageCond']:
    all_data[col]=all_data[col].fillna('None')

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [ ]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [ ]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)

In [ ]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [ ]:
all_data['Electrical']=all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [ ]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

In [ ]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

In [ ]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [ ]:
all_data_na=(all_data.isnull().sum())/len(all_data) * 100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
miss_data=pd.DataFrame({'Missing Ratio':all_data_na})
miss_data.head()

In [ ]:
all_data['MSSubClass'].dtype

In [ ]:
all_data['MSSubClass']=all_data['MSSubClass'].apply(str)

In [ ]:
all_data['MSSubClass'].dtype

In [ ]:
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

In [ ]:
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond',
        'ExterQual', 'ExterCond', 'HeatingQC', 'PoolQC', 'KitchenQual',
        'BsmtFinType1', 'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure',
        'GarageFinish', 'LandSlope', 'LotShape', 'PavedDrive', 'Street',
        'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')
for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c]=lbl.transform(list(all_data[c].values))

In [ ]:
all_data.shape

In [ ]:
all_data['TotalSF']=all_data['TotalBsmtSF']+all_data['1stFlrSF']+all_data['2ndFlrSF']

- skewed features

In [ ]:
numeric_feats=all_data.dtypes[all_data.dtypes != 'object'].index
skewed_feats=all_data[numeric_feats].apply(lambda x : skew(x.dropna())).sort_values(ascending=False)
skewness=pd.DataFrame({'Skew':skewed_feats})
skewness

- get abs(skew ) > 0.75

In [ ]:
skewness

In [ ]:
skewness=skewness[abs(skewness['Skew'])>0.75]

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox,boxcox_normmax

In [ ]:
for i in range(len(skewness)):
    print(skewness.index[i],'-----skew:',skewness.values[i],'-----lm:',boxcox_normmax(all_data[skewness.index[i]]+1),'kurt : ',all_data[skewness.index[i]].kurt())
    

- - change fixed lam=0.15 to different lam

In [ ]:
skewed_feats=skewness.index
for feat in skewed_feats:
    lm=boxcox_normmax(all_data[feat]+1)
    all_data[feat]=boxcox1p(all_data[feat],lm)
    print(feat,' skew : ', all_data[feat].skew(),' lm : ',lm,' kurt : ',lm,all_data[feat].kurt())

In [ ]:
all_data.head()

In [ ]:
all_data=pd.get_dummies(all_data)
all_data.shape

In [ ]:
train=all_data[:ntrain]
test=all_data[ntrain:]

#### Model

In [ ]:
from sklearn.linear_model import ElasticNet,Lasso,BayesianRidge,LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
n_folds=5

def rmsle_cv(model):
    kf=KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse=np.sqrt(-cross_val_score(model,train.values,y_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
lasso=make_pipeline(RobustScaler(),Lasso(alpha=0.0005,random_state=1))

In [ ]:
ridge=make_pipeline(RobustScaler(),BayesianRidge())

In [ ]:
lassoic=make_pipeline(RobustScaler(),LassoLarsIC())

In [ ]:
ENet=make_pipeline(RobustScaler(),ElasticNet(alpha=0.0005,l1_ratio=0.9,random_state=3))

In [ ]:
KRR=KernelRidge(alpha=0.6,kernel='polynomial',degree=2,coef0=2.5)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000,
                                   learning_rate=0.05,
                                   max_depth=4,
                                   max_features='sqrt',
                                   min_samples_leaf=15,
                                   min_samples_split=10,
                                   loss='huber',
                                   random_state=5)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score=rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score=rmsle_cv(ridge)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score=rmsle_cv(lassoic)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score=rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

### Stacked Model
- Simple stacking approach: average base models
- - Bast Stacked Model : Average base Models

In [ ]:
class AveragingModels(BaseEstimator,RegressorMixin,TransformerMixin):
    def __init__(self,models):
        self.models=models
    
    def fit(self,X,y):
        self.models_=[clone(x) for x in self.models]
        
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        predictions=np.column_stack([model.predict(X) for model in self.models_])
        return np.mean(predictions,axis=1)

In [ ]:
averaged_models=AveragingModels(models=(ENet,ridge,lassoic ,lasso))
score=rmsle_cv(averaged_models)
print('Averaged base models score : {:.4f} ({:.4f})\n'.format(score.mean(),score.std()))


- Blend average models

In [ ]:
class StackingAveragedModels(BaseEstimator,RegressorMixin,TransformerMixin):
    def __init__(self,base_models,meta_model,n_folds=5):
        self.base_models=base_models
        self.meta_model=meta_model
        self.n_folds=n_folds
    
    def fit(self,X,y):
        self.base_models_=[list() for x in self.base_models]
        self.meta_model_=clone(self.meta_model)
        kfold=KFold(n_splits=self.n_folds,shuffle=True,random_state=156)
        
        out_of_fold_predictions=np.zeros((X.shape[0],len(self.base_models)))
        for i ,model in enumerate(self.base_models):
            for train_index,holdout_index in kfold.split(X,y):
                instance=clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index],y[train_index])
                y_pred=instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index,i]=y_pred
        self.meta_model_.fit(out_of_fold_predictions,y)
        return self
    
    def predict(self,X):
        meta_features=np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models=StackingAveragedModels(base_models=(ENet,ridge,lassoic),meta_model=lasso)
score=rmsle_cv(stacked_averaged_models)
print('Stacking Averaged models score : {:.4f} ({:.4f})'.format(score.mean(),score.std()))


- Ensembling Stacked Regressor,XGBoost and LightGBM

In [ ]:
def rmsle(y,y_pred):
    return np.sqrt(mean_squared_error(y,y_pred))

In [ ]:
y_train.shape

In [ ]:
test.values.shape

In [ ]:
stacked_averaged_models.fit(train.values,y_train)
stacked_train_pred=stacked_averaged_models.predict(train.values)
stacked_pred=np.expm1(stacked_averaged_models.predict(test.values))
print(rmsle(y_train,stacked_train_pred))

In [ ]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))